# Advent of Code 2024

> If debugging is the process of removing software bugs, then programming must be the process of putting them in.

-- Edsger W. Dijkstra

## Imports and definitions

In [1]:
import re
from itertools import chain, islice, product
from functools import cache, reduce
from operator import add, mul
from urllib import request
from collections import Counter, defaultdict
from math import prod, inf

def aocin(day):
    try:
        with open(f'input/{day}') as f:
            return f.read().strip()
    except FileNotFoundError:
        r = request.Request(f'https://adventofcode.com/2024/day/{day}/input')
        r.add_header('Cookie', open('../.aoccookie').read().strip())
        r.add_header('User-Agent', 'github.com/edoannunziata/jardin')
        with open(f'input/{day}', 'bw') as f:
            f.write(request.urlopen(r).read())
        with open(f'input/{day}') as f:
            return f.read().strip()

## [Day 1: Historian Hysteria](https://adventofcode.com/2024/day/1)

In [2]:
a, b = [list(t) for t in zip(*[[*map(int, x.split())] for x in aocin(1).split('\n')])]
na, nb = Counter(a), Counter(b)

A = sum(abs(u-v) for u, v in zip(sorted(a), sorted(b)))
assert A == 1388114

A = sum(u * nu * nb[u] for u, nu in na.items())
assert A == 23529853

## [Day 2: Red-Nosed Reports](https://adventofcode.com/2024/day/2)

In [3]:
reports = [[*map(int, l.split())] for l in aocin(2).split('\n')]

def is_safe(l, allowed, tol=0):
    def _f(l, tol):
        if tol < 0: return False
        if len(l) < 2: return True
        a, b, *rest = l
        return (
            ((a is None or a-b in allowed) and _f([b] + rest, tol))
            or _f([a] + rest, tol-1)
        )
    return _f([None] + l, tol)


A = sum(is_safe(l, {1, 2, 3}) or is_safe(l, {-1, -2, -3}) for l in reports)
assert A == 369

A = sum(is_safe(l, {1, 2, 3}, tol=1) or is_safe(l, {-1, -2, -3}, tol=1) for l in reports)
assert A == 428

## [Day 3: Mull it Over](https://adventofcode.com/2024/day/3)

In [4]:
A = sum(
    prod(map(int, m.groups()))
    for m in re.finditer(r'mul\((\d{1,3}),(\d{1,3})\)', aocin(3))
)
assert A == 160672468


def mul_with_switch(input):
    active, acc = True, 0
    for m in re.finditer(r"(do|don't|mul)\((\d{0,3}),?(\d{0,3})\)", input):
        match m.groups():
            case "mul", _ as a, _ as b:
                acc += active * int(a) * int(b)
            case "do", _, _:
                active = True
            case "don't", _, _:
                active = False
    return acc

A = mul_with_switch(aocin(3))
assert A == 84893551

## [Day 4: Ceres Search](https://adventofcode.com/2024/day/4)

In [5]:
def gen_input(raw):
    rows = raw.split('\n')
    cols = [''.join(u) for u in list(zip(*rows))]
    diag_primary = defaultdict(str)
    diag_secondary = defaultdict(str)
    for i in range(len(rows)):
        for j in range(len(rows[0])):
            diag_primary[i-j] += rows[i][j]
            diag_secondary[i+j] += rows[i][j]
    return rows, cols, diag_primary.values(), diag_secondary.values()


rows, cols, diag_primary, diag_secondary = gen_input(aocin(4))
A = sum(
    u.count('XMAS') + u.count('SAMX')
    for u in chain(rows, cols, diag_primary, diag_secondary)
)
assert A == 2613

A = sum(
    (
            rows[i][j] == 'A'
        and {rows[i+1][j+1], rows[i-1][j-1]} == {'S', 'M'}
        and {rows[i+1][j-1], rows[i-1][j+1]} == {'S', 'M'} 
    )
    for i in range(1, len(rows) - 1)
    for j in range(1, len(rows[0]) - 1)
)
assert A == 1905

## [Day 5: Print Queue](https://adventofcode.com/2024/day/5)

The algorithms and data structures begin to strike! :)

It's easy to observe that the problem is asking questions related to the topological ordering of the graph of rules.

The second part of the problem is actually underspecified. The text of the problem seems to be assuming that there exists a unique topological order, but that is obviously not the case, for a trivial counterexample imagine an empty rule graph and a nonempty update array; any permutation of the array would be valid.

In [6]:
rules, updates = aocin(5).split('\n\n')
updates_lst = [[*map(int, r.split(','))] for r in updates.split('\n')]

def rules_graphs(rules):
    direct, reverse = defaultdict(set), defaultdict(set)
    for rule in rules.split('\n'):
        a, b = map(int, rule.split('|'))
        direct[b].add(a), reverse[a].add(b)
    return direct, reverse

def is_inorder(rules_graph, update):
    present, done = set(update), set()
    for u in update:
        if any(v in present and v not in done for v in rules_graph[u]): return False
        done.add(u)
    return True

def topological_sort(direct, reverse, update):
    present = set(update)
    out_degree = {k: sum(u in present for u in direct[k]) for k in present}
    active = set(k for k, v in out_degree.items() if v == 0) 
    while active:
        yield (x := active.pop())
        out_degree[x] = -1
        for v in reverse[x]:
            if v in out_degree: out_degree[v] -= 1
            if out_degree.get(v, -1) == 0: active.add(v)


direct, reverse = rules_graphs(rules)

A = sum(is_inorder(direct, u) * u[len(u) // 2] for u in updates_lst)
assert A == 4185

A = sum(
    next(islice(topological_sort(direct, reverse, u), len(u) // 2, len(u) // 2 + 1))
    for u in updates_lst if not is_inorder(direct, u)
)
assert A == 4480

## [Day 6 - Guard Gallivant](https://adventofcode.com/2024/day/6)

In [7]:
class Grid:
    def __init__(self, raw):
        self.rows = raw.split('\n')
        self.height = len(self.rows)
        self.width = len(self.rows[0])
        for i in range(self.height):
            for j in range(self.width):
                if self[i, j] == '^':
                    self.starting_point = (i, j)
                    break
                    
    def __getitem__(self, u):
        if (0 <= u[0] < self.height) and (0 <= u[1] < self.width):
            return self.rows[u[0]][u[1]]
        else: raise IndexError()


def get_visited(grid):
    i, j = grid.starting_point
    di, dj = -1, 0
    yield i, j
    while True:
        ni, nj = i + di, j + dj
        try:
            match grid[ni, nj]:
                case '#': di, dj = dj, -di
                case  _ : i, j = i+di, j+dj; yield i, j
        except IndexError: 
            break

@cache
def compute_next(grid, i, j, di, dj):
    pi, pj = i, j
    try:
        while grid[pi, pj] != '#':
            pi, pj = pi+di, pj+dj
        return pi-di, pj-dj
    except IndexError:
        return None, None

def generate_loops(grid):
    for h, k in set(get_visited(grid)):
        seen = set()
        i, j = grid.starting_point
        di, dj = -1, 0
        while True:
            if (i, j) == (None, None): break
            if (i, j, di, dj) in seen: yield h, k; break
            else: seen.add((i, j, di, dj))
            ni, nj = compute_next(grid, i, j, di, dj)
            if   (di, dj) == (0, 1)  and i == h and j <= k <= (nj or inf) : i, j = i, k-1
            elif (di, dj) == (0, -1) and i == h and (nj or -inf) <= k <= j: i, j = i, k+1
            elif (di, dj) == (1, 0)  and j == k and i <= h <= (ni or inf) : i, j = h-1, j
            elif (di, dj) == (-1, 0) and j == k and (ni or -inf) <= h < i : i, j = h+1, j
            else: i, j = ni, nj
            di, dj = dj, -di


A = len(set(get_visited(Grid(aocin(6)))))
assert A == 4939

A = sum(1 for _ in generate_loops(Grid(aocin(6))))
assert A == 1434

## [Day 7: Bridge Repair](https://adventofcode.com/2024/day/7)


In [8]:
def gen_lines(raw):
    lines = []
    for line in raw.split('\n'):
        target, ls = line.split(':')
        lines.append((int(target), [*map(int, ls.split())]))
    return lines


def can_result_in(target, elem, op):
    return target in reduce(
        lambda u, v: (
            {z for f, x in product(op, u) if (z := f(x, v)) <= target}
        ),
        elem[1:], {elem[0]} 
    )
   
op = {add, mul} 
A = sum(k for k, v in gen_lines(aocin(7)) if can_result_in(k, v, op))
assert A == 5030892084481

op = {add, mul, lambda a, b: int(str(a) + str(b))}
A = sum(k for k, v in gen_lines(aocin(7)) if can_result_in(k, v, op))
assert A == 91377448644679